In [1]:
import pandas as pd
import numpy as np
import re,json

import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.csv as pc
import pandas as pd

from collections import defaultdict

from sklearn.feature_extraction.text import TfidfVectorizer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 1. data확인

In [2]:
table = pc.read_csv("./prodct_name.tsv",parse_options=pc.ParseOptions(delimiter="\t"))
df=table.to_pandas()
df.head()

,id,name
0,153679680,생로랑 풀스톤네일
1,153680960,고정 걸쇠 브라켓
2,153682496,마인크래프트 자바에디션
3,153683520,남성의류 1번 코오롱반바지
4,153683776,기프티콘


In [3]:
df.shape
df.dtypes

(999943, 2)

id       int64
name    object
dtype: object

In [4]:
### null값 있는지 확인 : 없음

df.isnull().sum()

id      0
name    0
dtype: int64

### 2. Token 확인

In [5]:
def tokenizer(data):
    token=[]

    data = data.lower() #소문자로 변환
    words = data.split() #공백으로 분리
    #print("split ", words)

    #규칙에 해당 -> findall
    p = re.compile("[가-힣]+|[ㄱ-ㅎ|ㅏ-ㅣ]+|[a-z0-9-]+|[^ a-z0-9가-힣+]") #규칙

    for word in words:
        find = re.findall(p,word)
        for w in find:
            #token=token+w+' '
            token.append(w)
    return token

In [138]:
query = "아이폰8"
test1 = "주술회전 ㅎㅅㅈ 학생 증 판매"
test2 = "7번 라이젠2600 데스크탑 컴퓨터 본체PC 판매합니다"
test3 = "블랙 메리제인[230/5cm]"
test4 = "갤럭시s10 5g 255G 크라운실버 내용확인/19만원"

In [163]:
tokenizer(query)
tokenizer(test1)
tokenizer(test2)
tokenizer(test3)
tokenizer(test4)

'아이폰\n8\n'

'주술회전\nㅎㅅㅈ\n학생\n증\n판매\n'

'7\n번\n라이젠\n2600\n데스크탑\n컴퓨터\n본체\npc\n판매합니다\n'

'블랙\n메리제인\n[\n230\n/\n5cm\n]\n'

'갤럭시\ns10\n5g\n255g\n크라운실버\n내용확인\n/\n19\n만원\n'

In [6]:
df['token'] = df['name'].apply(tokenizer)
df.tail()

,id,name,token
999938,154650538,핑크에이지 오픈반가발 브리즈펌 아이보리골드,"[핑크에이지, 오픈반가발, 브리즈펌, 아이보리골드]"
999939,154650539,연예인들이 많이 입는 theory 수트셋업,"[연예인들이, 많이, 입는, theory, 수트셋업]"
999940,154650542,갤럭시S10 대량.소량 팝니다 중고폰 s10,"[갤럭시, s10, 대량, ., 소량, 팝니다, 중고폰, s10]"
999941,154650548,SSAMZIE 지갑,"[ssamzie, 지갑]"
999942,154650551,갤럭시북 이온2 NT950XDZ-G58AW 미개봉,"[갤럭시북, 이온, 2, nt950xdz-g58aw, 미개봉]"


## 3. invert inde파일 생성

In [7]:
js = df.to_json(orient = 'records')
json_data =json.loads(js)

In [9]:

index_dict=defaultdict(list)

for data in json_data:
    for token in data['token']:
            index_dict[token].append(data['id'])

p=pd.DataFrame(list(index_dict.items()),columns=['token', 'docu_list'])

In [10]:
table = pa.Table.from_pandas(p)
pq.write_table(table, './data/real_index.parquet')

In [11]:
index_df = pq.read_table('./data/real_index.parquet').to_pandas()
index_df.head()

,token,docu_list
0,생로랑,"[153679680, 153883200, 153854475, 153932820, 1..."
1,풀스톤네일,"[153679680, 153681317]"
2,고정,"[153680960, 153957133, 154060025, 154167619, 1..."
3,걸쇠,"[153680960, 153681055, 153681628]"
4,브라켓,"[153680960, 153682720, 153892892, 153774321, 1..."


## 4. tf-idf 파일생성

In [77]:
## token 많아질수록 벡터의 차원이 커지는 문제
## 가장 많이 나온 단어 n개만 사용하는 max_features 파라미터 : TfidfVectorizer(max_features=4)
## all_teokn = tmp['token ] : list

vect2 = TfidfVectorizer(tokenizer=tokenizer,max_features=100)
tfvect_matrix = vect2.fit_transform(df['name'])
#tfvect_matrix.toarray()

tfidf_col = vect2.get_feature_names() 

tfidv_df = pd.DataFrame(tfvect_matrix.toarray(), index = list(df['id']), columns = sorted(tfidf_col))
#tfidv_df.head()

### 전체데이터로 tf-idf생성시 mastrix이 너무 커서 메모리 이슈 발생
- 가장 많이 나온 단어 n개만 사용하는 max_features 파라미터 : TfidfVectorizer(max_features=4)

In [ ]:
table = pa.Table.from_pandas(tfidv_df)
pq.write_table(table, './data/real_tfidv.parquet')

In [64]:
tfidv_df_file = pq.read_table('./data/real_tfidv.parquet').to_pandas()
#tfidv_df_file.head()

,!,#,%,(,),*,",",-,.,/,...,티셔츠,판매,판매합니다,팔아요,팝니다,포카,폴로,프로,플러스,화이트
153679680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
153680960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
153682496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
153683520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
153683776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 5. 검색어 테스트

In [122]:
### 1. token_list : tokenizer에 의해 분리

In [160]:
q = "맥북m1 13인치 미개봉"
#입력한 키워드 토큰화 
token_list = tokenizer(q)
print(token_list)

['맥북', 'm1', '13', '인치', '미개봉']


In [161]:
### 2. new_token_list : token_list 중에서 index_df에 검색한 token list

In [162]:
# 0831 : ndex_dict 파일로 바꾼 ver - 없는 키워드 입력 고려 

search_token =index_df[index_df['token'].isin(token_list)]
#print(search_token)

new_token_list = list(search_token['token'])
print(new_token_list) 

if len(new_token_list) == 0:
    print("없는 키워드")

q_documents=[]
for docu_list in search_token['docu_list']:
    q_documents.append(set(docu_list))

# 문서들의 교집합 
query_documents = list(q_documents[0].intersection(*q_documents))
print(len(query_documents))

if len(query_documents) == 0:
    print("교집합없음")

['인치', '미개봉', '13', 'm1', '맥북']
11


In [163]:
new_token_list

#교집합 문서들의 id,name
search_dc = df[df['id'].isin(query_documents)] #id기준(행기준) 검색
search_dc=search_dc.set_index('id')

search_dc.head()

['인치', '미개봉', '13', 'm1', '맥북']

,name,token
id,,
153957513,맥북 프로 13인치 m1 512gb 스그 미개봉 새상품 판매합니다,"[맥북, 프로, 13, 인치, m1, 512gb, 스그, 미개봉, 새상품, 판매합니다]"
153911702,미개봉/ 애플 맥북 에어 13인치 M1 스페이스 그레이 256g 최저가,"[미개봉, /, 애플, 맥북, 에어, 13, 인치, m1, 스페이스, 그레이, 25..."
153767101,(미개봉) 맥북 프로 2020 M1칩 256기가 13인치 팝니다,"[(, 미개봉, ), 맥북, 프로, 2020, m1, 칩, 256, 기가, 13, ..."
154330471,맥북 프로 m1 13인치 기본형 미개봉,"[맥북, 프로, m1, 13, 인치, 기본형, 미개봉]"
154420300,맥북 프로 m1 13인치 램16GB 미개봉 삽니다,"[맥북, 프로, m1, 13, 인치, 램, 16gb, 미개봉, 삽니다]"


### 검색한 키워드(new_token_list)중에 tf-idf의 컬럼으로 안들어있는게 있을때 문제
- 예를 들어, 검색한 키워드는 ['맨투맨', '톰브라운', '정품'] 
- 그런데 TfidfVectorizer옵션에서 메모리이슈문제로 feature를 100개만 뽑다보니 없는 키워드가 존재 
- KeyError: "['톰브라운'] not in index"
<br><br>
- tfidv_df의 column로 현재 feature에 있는 키워드 list(new_token_list2)생성
- tfidv_df에서 검색

In [164]:
### 3. new_token_list2 : new_token_list 중에서 tfidv_df 검색한 token list

In [165]:
#[tfidv_df.loc[query_documents][i] for i in new_token_list if i in tfidv_df.columns]

#교집합 문서들에 대해서 tf-dif(score값)
#tfidv_df.loc[query_documents] #교집합문서
new_token_list2 =[]
for token in new_token_list:
    if token in tfidv_df.columns:
        new_token_list2.append(token)

search_tf = tfidv_df.loc[query_documents][new_token_list2] ## 여기에서 token_list에서 tokne에 없는게 있음 

search_tf['score'] = search_tf.sum(axis=1)
search_tf.head()

,인치,미개봉,score
154638051,0.715301,0.698816,1.414117
154330471,0.560861,0.547935,1.108796
154637224,0.500556,0.489020,0.989576
153957513,0.467213,0.456445,0.923658
154420300,0.475147,0.464196,0.939343


In [166]:

# search_dc와 search_tf join (by id)
search=search_tf.join(search_dc,how='inner')
search['pid']=search.index

search.sort_values(by=['score'],ascending=[False],inplace=True) #score기준 정렬
search.head()

,인치,미개봉,score,name,token,pid
154638051,0.715301,0.698816,1.414117,M1 맥북 에어 13인치 기본형 256GB 스페이스그레이 미개봉,"[m1, 맥북, 에어, 13, 인치, 기본형, 256gb, 스페이스그레이, 미개봉]",154638051
153911702,0.623155,0.608794,1.231948,미개봉/ 애플 맥북 에어 13인치 M1 스페이스 그레이 256g 최저가,"[미개봉, /, 애플, 맥북, 에어, 13, 인치, m1, 스페이스, 그레이, 25...",153911702
154330471,0.560861,0.547935,1.108796,맥북 프로 m1 13인치 기본형 미개봉,"[맥북, 프로, m1, 13, 인치, 기본형, 미개봉]",154330471
154450284,0.560861,0.547935,1.108796,맥북 프로 13인치 M1 512G 2020년형 미개봉,"[맥북, 프로, 13, 인치, m1, 512g, 2020, 년형, 미개봉]",154450284
154672987,0.560861,0.547935,1.108796,맥북 프로 M1 13인치 512GB 스그 미개봉,"[맥북, 프로, m1, 13, 인치, 512gb, 스그, 미개봉]",154672987


In [167]:
# 최종결과 
response = search[['pid','name','score']]
response.head()

# response msg
js = response.to_json(orient='records')
res_data =json.loads(js)
res_data

,pid,name,score
154638051,154638051,M1 맥북 에어 13인치 기본형 256GB 스페이스그레이 미개봉,1.414117
153911702,153911702,미개봉/ 애플 맥북 에어 13인치 M1 스페이스 그레이 256g 최저가,1.231948
154330471,154330471,맥북 프로 m1 13인치 기본형 미개봉,1.108796
154450284,154450284,맥북 프로 13인치 M1 512G 2020년형 미개봉,1.108796
154672987,154672987,맥북 프로 M1 13인치 512GB 스그 미개봉,1.108796


[{'pid': 154638051,
  'name': 'M1 맥북 에어 13인치 기본형 256GB 스페이스그레이 미개봉',
  'score': 1.4141174789},
 {'pid': 153911702,
  'name': '미개봉/ 애플 맥북 에어 13인치 M1 스페이스 그레이 256g 최저가',
  'score': 1.2319484653},
 {'pid': 154330471, 'name': '맥북 프로 m1 13인치 기본형 미개봉', 'score': 1.1087962006},
 {'pid': 154450284,
  'name': '맥북 프로 13인치 M1 512G 2020년형 미개봉',
  'score': 1.1087962006},
 {'pid': 154672987,
  'name': '맥북 프로 M1 13인치 512GB 스그 미개봉',
  'score': 1.1087962006},
 {'pid': 154637224,
  'name': '맥북 프로 M1 13인치 2020년형 512G 그레이 미개봉 ! 부산중고',
  'score': 0.9895757488},
 {'pid': 154420300,
  'name': '맥북 프로 m1 13인치 램16GB 미개봉 삽니다',
  'score': 0.9393429483},
 {'pid': 153957513,
  'name': '맥북 프로 13인치 m1 512gb 스그 미개봉 새상품 판매합니다',
  'score': 0.9236575966},
 {'pid': 154047517,
  'name': '맥북 프로 13인치 M1 8gb 512GB 스그 미개봉 새상품 판매합니다',
  'score': 0.9236575966},
 {'pid': 153767474,
  'name': '(미개봉) 맥북 프로 2020 M1칩 256기가 13인치 팝니다',
  'score': 0.8407174596},
 {'pid': 153767101,
  'name': '(미개봉) 맥북 프로 2020 M1칩 256기가 13인치 팝니다',
  'scor